# Extracting Crop EVI from Kenyan Towns

We extract Crop EVI from: Nairobi (NRB), Makueni (MAK), Kakamega (KAK) and Wajir (WAJ)

**Products used:** 
* [ls7_sr](https://explorer.digitalearth.africa/ls7_sr)
* [ls8_sr](https://explorer.digitalearth.africa/ls8_sr)
* [ls9_sr](https://explorer.digitalearth.africa/ls9_sr)


## Background
[Enhanced Vegetation Index](https://shorturl.at/oHRY9) can be calculated from Landsat or Sentinel-2 images, and is similar to the Normalized Difference Vegetation Index (NDVI), as it quantifies vegetation greenness. However, the EVI corrects for some atmospheric conditions and canopy background noise and is more sensitive in areas with dense vegetation.

Using Digital Earth Africa's archive of analysis-ready satellite data, we can easily calculate the EVI for mapping and monitoring vegetation through time, or as inputs to machine learning or classification algorithms.

## Description
This notebook demonstrates how to:

1. Load Landsat 7 images for an area of interest (AOI)
2. Calculate the Enhanced Vegetation Index (EVI)
3. Compute average EVA and save results in CSV file
4. Visualize the results.

***


### Import Libraries

In [1]:
%matplotlib inline

import datacube
import xarray as xr
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from datacube.utils.geometry import Geometry

from deafrica_tools.datahandling import load_ard, mostcommon_crs
from deafrica_tools.plotting import rgb
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.areaofinterest import define_area

### Connect to the datacube

In [2]:
dc = datacube.Datacube(app='crop_health_evi')

### Analysis parameters

The following cell sets the parameters, which define the area of interest and the length of time to conduct the analysis over.
The parameters are

* `lat`: The central latitude to analyse (e.g. `10.338`).
* `lon`: The central longitude to analyse (e.g. `-1.055`).
* `buffer`: The number of square degrees to load around the central latitude and longitude.
For reasonable loading times, set this as `0.1` or lower.


#### Select location
To define the area of interest, there are two methods available:

1. By specifying the latitude, longitude, and buffer. This method requires you to input the central latitude, central longitude, and the buffer value in square degrees around the center point you want to analyze. For example, `lat = 10.338`, `lon = -1.055`, and `buffer = 0.1` will select an area with a radius of 0.1 square degrees around the point with coordinates (10.338, -1.055).

2. By uploading a polygon as a `GeoJSON or Esri Shapefile`. If you choose this option, you will need to upload the geojson or ESRI shapefile into the Sandbox using Upload Files button <img align="top" src="../Supplementary_data/upload_files_icon.png"> in the top left corner of the Jupyter Notebook interface. ESRI shapefiles must be uploaded with all the related files `(.cpg, .dbf, .shp, .shx)`. Once uploaded, you can use the shapefile or geojson to define the area of interest. Remember to update the code to call the file you have uploaded.

To use one of these methods, you can uncomment the relevant line of code and comment out the other one. To comment out a line, add the `"#"` symbol before the code you want to comment out. By default, the first option which defines the location using latitude, longitude, and buffer is being used.

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.

In [3]:
# Method 1: Specify the latitude, longitude, and buffer
aoi = define_area(lat=-1.319456, lon=36.852516, buffer=0.02)  # NRB
#aoi = define_area(lat=3.355112, lon=39.738751, buffer=0.02)  # WAJ
#aoi = define_area(lat=0.292923, lon=34.759233, buffer=0.02)  # KAK
#aoi = define_area(lat=-2.282646, lon=31.41094, buffer=0.02)  # MAK

#Create a geopolygon and geodataframe of the area of interest
geopolygon = Geometry(aoi["features"][0]["geometry"], crs="epsg:4326")
geopolygon_gdf = gpd.GeoDataFrame(geometry=[geopolygon], crs=geopolygon.crs)

# Get the latitude and longitude range of the geopolygon
lat_range = (geopolygon_gdf.total_bounds[1], geopolygon_gdf.total_bounds[3])
lon_range = (geopolygon_gdf.total_bounds[0], geopolygon_gdf.total_bounds[2])

## Create a query and load satellite data

The `load_ard` function will automatically mask out clouds from the dataset, allowing us to focus on pixels that contain useful data.
It will also exclude images where more than 99% of the pixels are masked, which is set using the `min_gooddata` parameter in the `load_ard` call.

In [4]:
# Create a reusable query
query = {
    'x': lon_range,
    'y': lat_range,
    'time': ('2000-01-01', '2022-12-31'),
    'resolution': (-10, 10)
}

# Identify the most common projection system in the input query
# product = 'ls7_sr'
# product = 'ls8_sr'
# product = 'ls9_sr'
output_crs = mostcommon_crs(dc=dc, product='ls7_sr', query=query)

# Load available data from Landsat 9 and filter to retain only times
# with at least 99% good data
ds = load_ard(dc=dc,
              products=['ls7_sr'],
              min_gooddata=0.99,
              measurements=['red', 'green', 'blue', 'nir'],
              output_crs=output_crs,
              **query)

/opt/venv/lib/python3.10/site-packages/deafrica_tools/datahandling.py:739: UserWarning: Multiple UTM zones ['epsg:32637', 'epsg:32636'] were returned for this query. Defaulting to the most common zone: epsg:32637
  warnings.warn(


Using pixel quality parameters for USGS Collection 2
Finding datasets
    ls7_sr
Counting good quality pixels for each time step


/opt/venv/lib/python3.10/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(


Filtering to 54 out of 327 time steps with at least 99.0% good quality pixels
Applying pixel quality/cloud mask
Re-scaling Landsat C2 data
Loading 54 time steps


In [5]:
# Calculate EVI using `calculate indices`
ds = calculate_indices(ds, index='EVI', satellite_mission='ls')

#The vegetation proxy index should now appear as a data variable,
#along with the loaded measurements, in the `ds` object.


In [6]:

# Assuming 'ds' is your dataset with EVI values over time, loaded using calculate_indices
# And it contains a 'time' dimension and 'EVI' data variable

# 1. Compute the mean EVI for each time point
evi_avg_over_time = ds['EVI'].mean(dim=['x', 'y'])  # Adjust dims to match your spatial dimensions

# 2. Convert to a Pandas DataFrame
evi_avg_df = evi_avg_over_time.to_dataframe().reset_index()

# 3. Rename columns and format data as required
evi_avg_df = evi_avg_df[['time', 'EVI']]  # Select only time and EVI columns
evi_avg_df.columns = ['date', 'EVI']  # Rename columns

# 4. Save to CSV
evi_avg_df.to_csv("average_evi_nrb.csv", index=False)
#evi_avg_df.to_csv("average_evi_mak.csv", index=False)
#evi_avg_df.to_csv("average_evi_waj.csv", index=False)
#evi_avg_df.to_csv("average_evi_kak.csv", index=False)
print("Saved average EVI data to 'average_evi.csv'")

# evi_avg_df

Saved average EVI data to 'average_evi.csv'


In [7]:
# Plot the selected EVI results at timestep 1, 10, 15 and last image
#ds.isel(time=[1, 10, 15, -1]).EVI.plot(col='time',
#                                       cmap='RdYlGn',
#                                       size=6,
#                                       col_wrap=2)